In [1]:
import pandas as pd

# === Step 1: Load the Excel file
df = pd.read_excel(r"C:\Users\preeti123\Desktop\PREETI\carbon footprint\Transactions_dataset.xlsx")

# === Step 2: Define CO₂ emission factors
emission_factors = {
    ("Food", "Dairy"): {"cost_per_unit": 150, "co2e_per_unit": 3.0},
    ("Food", "Non-veg"): {"cost_per_unit": 350, "co2e_per_unit": 10.0},
    
    ("Groceries", "Packaged"): {"cost_per_unit": 150, "co2e_per_unit": 1.0},

    ("Shopping", "Online"): {"cost_per_unit": 700, "co2e_per_unit": 2.5},
    ("Shopping", "Offline"): {"cost_per_unit": 700, "co2e_per_unit": 1.8},

    ("Utilities", "Electricity"): {"cost_per_unit": 8, "co2e_per_unit": 0.5},

    ("Transport", "Taxi"): {"cost_per_unit": 15, "co2e_per_unit": 0.2},
    ("Transport", "Public Transport"): {"cost_per_unit": 2.5, "co2e_per_unit": 0.05},

    ("Fuel & Energy", "Diesel"): {"cost_per_unit": 90, "co2e_per_unit": 2.68},
    ("Fuel & Energy", "Petrol"): {"cost_per_unit": 100, "co2e_per_unit": 2.31},
    ("Fuel & Energy", "LPG"): {"cost_per_unit": 1100, "co2e_per_unit": 42.6},

    ("Travel", "Flight"): {"cost_per_unit": 6, "co2e_per_unit": 0.22},
    ("Travel", "Train"): {"cost_per_unit": 1.5, "co2e_per_unit": 0.05},
    ("Travel", "Hotel"): {"cost_per_unit": 2500, "co2e_per_unit": 15.0},

}

# === Step 3: Function to calculate CO₂ emission ===
def calculate_emission(row):
    key = (row['Category'], row['Subcategory'])
    if key in emission_factors:
        factor = emission_factors[key]
        try:
            qty = row['Amount'] / factor['cost_per_unit']
            return round(qty * factor['co2e_per_unit'], 3)
        except:
            return None
    else:
        return 0

# === Step 4: Apply function to dataset ===
df["Estimated_CO2e"] = df.apply(calculate_emission, axis=1)

# === Step 5: Save to a specific path on your PC ===
# 💡 Change this to where YOU want to save the file:
output_path = r"C:\Users\preeti123\Desktop\PREETI\carbon footprint\Transactions_dataset_with_CO2.xlsx"

df.to_excel(output_path, index=False)

print(f"✅ File saved successfully at: {output_path}")


✅ File saved successfully at: C:\Users\preeti123\Desktop\PREETI\carbon footprint\Transactions_dataset_with_CO2.xlsx


In [3]:
import pandas as pd

# Step 1: Load your Excel data
df = pd.read_excel(r"C:\Users\preeti123\Desktop\PREETI\carbon footprint\Transactions_dataset_with_CO2.xlsx")

# Step 2: Create user summary
user_summary = df.groupby("User_ID").agg(
    Total_Transactions=("Transaction_ID", "count"),
    Total_Amount=("Amount", "sum"),
    Income=("Income", "first"),
    User_Type=("User_Type", "first"),
    Total_CO2e=("Estimated_CO2e", "sum")
)

# Step 3: Filter transactions with Eco_status == 'Eco'
eco_transactions = df[df["Eco_status"] == "Eco"]

# Step 4: Calculate Eco Spend
eco_spend = eco_transactions.groupby("User_ID").agg(
    Eco_Spend=("Amount", "sum")
)

# Step 5: Merge and calculate % Eco Spend
user_summary = user_summary.merge(eco_spend, how="left", on="User_ID").fillna(0)
user_summary["% Eco Spend"] = (user_summary["Eco_Spend"] / user_summary["Total_Amount"]) * 100
user_summary = user_summary.drop(columns=["Eco_Spend"]).reset_index()

# Rename columns
user_summary.columns = [
    "User_ID", "Total_Transactions", "Total_Amount", "Income",
    "User_Type", "Total_CO2e", "% Eco Spend"
]

# Step 6: Create Excel file with multiple sheets
output_path = r"C:\Users\preeti123\Desktop\PREETI\carbon footprint\Final_User_CO2e_Summary.xlsx"

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    # Sheet 1: All users
    user_summary.to_excel(writer, sheet_name="All Users", index=False)
    
    # Sheet 2: Total_CO2e < 120
    user_summary[user_summary["Total_CO2e"] < 120].to_excel(writer, sheet_name="CO2e < 120", index=False)
    
    # Sheet 3: 120 <= Total_CO2e <= 250
    user_summary[(user_summary["Total_CO2e"] >= 120) & (user_summary["Total_CO2e"] <= 250)] \
        .to_excel(writer, sheet_name="120 <= CO2e <= 250", index=False)
    
    # Sheet 4: Total_CO2e > 250
    user_summary[user_summary["Total_CO2e"] > 250].to_excel(writer, sheet_name="CO2e > 250", index=False)
    
    # Sheet 5: % Eco Spend > 60
    user_summary[user_summary["% Eco Spend"] > 60].to_excel(writer, sheet_name="% EcoSpend > 60", index=False)
    
    # Sheet 6: 40 <= % Eco Spend <= 60
    user_summary[(user_summary["% Eco Spend"] >= 40) & (user_summary["% Eco Spend"] <= 60)] \
        .to_excel(writer, sheet_name="40 <= % EcoSpend <= 60", index=False)
    
    # Sheet 7: % Eco Spend < 40
    user_summary[user_summary["% Eco Spend"] < 40].to_excel(writer, sheet_name="% EcoSpend < 40", index=False)
    
    # Sheet 8: % Eco Spend > 60 AND Total_CO2e < 120
    user_summary[(user_summary["% Eco Spend"] > 60) & (user_summary["Total_CO2e"] < 120)] \
        .to_excel(writer, sheet_name="Eligible for Green offers", index=False)

print(f"✅ File saved successfully at: {output_path}")

✅ File saved successfully at: C:\Users\preeti123\Desktop\PREETI\carbon footprint\Final_User_CO2e_Summary.xlsx
